In [28]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Styles
colmap = {
     "CLOSED" : "limegreen",
     "STOPLOSS" : "red",
     "OPEN" : "royalblue"
 }

barmap = {
     "BULL" : "limegreen",
     "BEAR" : "red",
     "RANGE" : "lightblue",
     "UNKNOWN" : "lightgrey"
 }

linemap = {
     "BUSD liquid" : "lightgreen",
     "Asset price" : "lightblue",
     "Portfolio value" : "orange"
 }

In [21]:
# Load in all data
busd = pd.read_json("./busdOverTime.json")
trades = pd.read_json("./backtestOrderbook.json")
market = pd.read_json("./marketData.json")
portfolioValueOverTime = pd.read_json("./walletValueOverTime.json")

In [64]:
# Build graph
fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])

# AVAILABLE BUSD
fig.add_trace(
    go.Scatter(
        x = busd["timestamp"],
        y = busd["value"], mode = "lines",
        name = "BUSD liquid",
        marker = dict(color = linemap["BUSD liquid"])
    ), secondary_y = False)

# MARKET PRICE
fig.add_trace(
    go.Scatter(
        x = market["timeStamp"],
        y = market["close"], mode = "lines",
        name = "Asset price",
        marker =  dict(color = linemap["Asset price"])
    ), secondary_y = True)

# PORTFOLIO VALUE
fig.add_trace(
    go.Scatter(
        x = portfolioValueOverTime["timestamp"],
        y = portfolioValueOverTime["value"], mode = "lines",
        name = "Portfolio value",
        marker = dict(color = linemap["Portfolio value"])
    ), secondary_y = False)


# ORDERS
fig.add_trace(
        go.Scatter(
            x = trades["timestampBuy"],
            y = trades["priceBuy"],
            mode = "markers",
            name = "OPEN",
            marker = dict(color = colmap["OPEN"])
        ), secondary_y = True)

for s in trades["status"].unique():
    if s != "OPEN":
        fig.add_trace(
            go.Scatter(
                x= trades["timestampSell"],
                y= trades["priceSell"],
                mode = "markers",
                name = s,
                marker = dict(
                        color = colmap[s]
                    )
            ), secondary_y = True)

In [65]:
market["bar"] = max(market["close"]) * 1.1
uniqueMarketConditions = market["ADX_TREND_SHORT"].unique()

In [66]:
for b in uniqueMarketConditions:
    filteredSignalsBars = market[market["ADX_TREND_SHORT"] == b].copy()
    fig.add_trace(
            go.Bar(
                x = filteredSignalsBars["timeStamp"],
                y = filteredSignalsBars["bar"],
                name = b,
                width = 10,
                marker = dict(color = barmap[b])),

            )
    

In [67]:
fig.show()

In [38]:
uniqueMarketConditions

array(['UNKNOWN', 'BULL', 'BEAR', 'RANGE'], dtype=object)

In [48]:
filteredSignalsBars.head()

,close,high,low,MACD,STOCH,ROC,MA_SHORT,MA_LONG,ADX_PDI_SHORT,ADX_MDI_SHORT,ADX_RESULT_SHORT,ADX_TREND_SHORT,RSI,RSI_CROSS,timeStamp,asset,_id,SELLSIGNALS,BUYSIGNALS,bar
85,1750.08,1761.26,1744.05,NO_CHANGE,16.18675,0.01,1772.08,1782.77,0.1969,0.2245,19.8456,RANGE,44.4434,NONE,2023-03-25 04:00:00,ETH,n6rjH13WGTbGuRj0,"{'RSI': False, 'STOCH': False}",{'STOCH': True},2331.153
86,1747.26,1752.63,1738.67,NO_CHANGE,14.02135,-0.01,1762.07,1783.20,0.1909,0.2294,19.0813,RANGE,43.8913,NONE,2023-03-25 08:00:00,ETH,Cyic8OsUyphJWxEg,"{'RSI': False, 'STOCH': False}",{'STOCH': True},2331.153
87,1756.86,1765.14,1741.05,CROSSED_TO_NEGATIVE,21.39292,-0.03,1757.11,1783.30,0.2085,0.2171,17.8628,RANGE,46.3355,NONE,2023-03-25 12:00:00,ETH,UFywJQou6OSUdn9y,"{'RSI': False, 'STOCH': False}",{'STOCH': False},2331.153
88,1719.37,1759.59,1717.62,CROSSED_TO_POSITIVE,1.23579,-0.04,1749.97,1780.43,0.1895,0.2482,17.5453,RANGE,39.1609,NONE,2023-03-25 16:00:00,ETH,ZjNiWPUT2MXqch1v,"{'RSI': False, 'STOCH': False}",{'STOCH': True},2331.153
89,1744.01,1745.28,1715.86,CROSSED_TO_NEGATIVE,21.79805,-0.03,1746.84,1773.47,0.1773,0.2361,17.3082,RANGE,45.1701,NONE,2023-03-25 20:00:00,ETH,hZAKNwqqCt6PHUXK,"{'RSI': False, 'STOCH': False}",{'STOCH': False},2331.153


In [46]:
portfolioValueOverTime

,timestamp,value
0,2023-03-11 00:00:00,1000.000000
1,2023-03-11 04:00:00,1000.000000
2,2023-03-11 08:00:00,1000.000000
3,2023-03-11 12:00:00,1000.000000
4,2023-03-11 16:00:00,1000.000000
...,...,...
995,2023-08-23 20:00:00,1110.250361
996,2023-08-24 00:00:00,1108.148452
997,2023-08-24 04:00:00,1107.582767
998,2023-08-24 08:00:00,1103.711707


In [3]:
fig = go.Figure()
fig = make_subplots()

for a in signals.asset.unique():
    priceAt0 = signals[(signals.timestamp == 1) & ((signals.asset == a))].price.values[0]
    z = valueAt0 / priceAt0
    filteredSignals = signals[signals["asset"] == a]
    
    fig.add_trace(
        go.Scatter(
            x=filteredSignals["timestamp"],
            y=filteredSignals["price"] * z, mode = "lines",
            name = "Price of " + a + " (normalized)"
            
    ))
fig.add_trace(
    go.Scatter(
        x=value["timestamp"],
        y=value["value"], mode = "lines",
        name = "Wallet value over time",
        marker = dict(color = "orange")
    ), secondary_y = False)

fig.show()

In [9]:
0.0090 / 0.8500

0.010588235294117647

In [10]:
0.1104 / 21

0.005257142857142857

In [61]:
-4.06 / 2599.26

-0.0015619830259381514

In [40]:
import time
print(time.time())
delta = time.time() - (900000 * 500)
print(time.time() - delta)

1648634575.3766937
450000000.0


In [47]:
def printAndFetch(n):
    if (n - 500 >= 0):
        print(n)
        n -= 500
        printAndFetch(n)

In [48]:
n = 1300
printAndFetch(n)

1300
800
